In [2]:
import tatqa_utils
import pandas as pd

In [3]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [4]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [5]:
import tatqa_utils
import pandas as pd
import requests

In [6]:
model_name = "llama-3.2-3b-instruct"
def infer(triton_url, prompt):

        payload = {
            "text_input": prompt,
            "max_tokens": 500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
        )
        res = ret.json()
        print(res)
        query_response = res["text_output"]
        return query_response
    
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", "HI")

{'model_name': 'llama-3.2-3b-instruct', 'model_version': '1', 'text_output': 'HI-TECH\nThe term "hi-tech" is a shortened form of "high technology." It refers to the use of advanced technology in various fields, including science, engineering, and industry. Hi-tech products and services are often characterized by their innovative and sophisticated features, which can improve efficiency, productivity, and quality of life.\n\nExamples of hi-tech products and services include:\n\n1. Smartphones and other mobile devices with advanced features like GPS, Wi-Fi, and high-speed internet connectivity.\n2. Computers and laptops with high-performance processors, large storage capacity, and advanced graphics capabilities.\n3. Medical devices and equipment, such as MRI machines, CT scanners, and robotic surgery systems.\n4. Telecommunications equipment, including fiber optic cables, satellite communications systems, and high-speed internet networks.\n5. Advanced manufacturing technologies, such as 3

'HI-TECH\nThe term "hi-tech" is a shortened form of "high technology." It refers to the use of advanced technology in various fields, including science, engineering, and industry. Hi-tech products and services are often characterized by their innovative and sophisticated features, which can improve efficiency, productivity, and quality of life.\n\nExamples of hi-tech products and services include:\n\n1. Smartphones and other mobile devices with advanced features like GPS, Wi-Fi, and high-speed internet connectivity.\n2. Computers and laptops with high-performance processors, large storage capacity, and advanced graphics capabilities.\n3. Medical devices and equipment, such as MRI machines, CT scanners, and robotic surgery systems.\n4. Telecommunications equipment, including fiber optic cables, satellite communications systems, and high-speed internet networks.\n5. Advanced manufacturing technologies, such as 3D printing, nanotechnology, and robotics.\n\nThe term "hi-tech" is often used

In [12]:
system_prompt = "You are an expert on answering questions using an external data table, and arithmetic calculations. You receive a context as a TABLE and a QUESTION. In TABLE numeric values usually have commas as thousand separators. You have to generate a python code, without sample data, which can answer the question. Your response MUST contain only one function called  'run', and no wrapping class.  The function MUST return results with format (value, scale). You ARE NOT ALLOWED to write ANY other words."
        
def gen_code_infer(model_url, question, table):
    #prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    query = f"Given the following TABLE: {table}, answer the following QUESTION: {question}"
    prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    print(prompt)    
    res = infer(model_url, prompt)
    #res = llm.invoke(prompt)
    #print(res)#
    code = res.replace('```python','').replace('```','')
    code = code.replace('Answer:','')
    code = code.replace(prompt,'').strip()
    
    print("S", code, "@")
    return code
    
def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
code = gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)
exec_code(code, table)

<|start_header_id|>system<|end_header_id|>You are an expert on answering questions using an external data table, and arithmetic calculations. You receive a context as a TABLE and a QUESTION. In TABLE numeric values usually have commas as thousand separators. You have to generate a python code, without sample data, which can answer the question. Your response MUST contain only one function called  'run', and no wrapping class.  The function MUST return results with format (value, scale). You ARE NOT ALLOWED to write ANY other words.<|eot_id|><|start_header_id|>user<|end_header_id|>Given the following TABLE: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '

(7.367680675224196, 2)

In [19]:
item = devdf.iloc[0]
table = item['table']['table']
question =  item['questions'][4] ['question'] 
code = gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)
exec_code(code, table)

<|start_header_id|>system<|end_header_id|>You are an expert on answering questions using an external data table, and arithmetic calculations. You receive a context as a TABLE and a QUESTION. In TABLE numeric values usually have commas as thousand separators. You have to generate a python code, without sample data, which can answer the question. Your response MUST contain only one function called  'run', and no wrapping class.  The function MUST return results with format (value, scale). You ARE NOT ALLOWED to write ANY other words.<|eot_id|><|start_header_id|>user<|end_header_id|>Given the following TABLE: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']], answer the following QUESTION: What is the change in Other in 2019 from 2018?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{'model_name': 'llama-3.2-3b-in

(-12.600000000000001, 'Decrease')

SyntaxError: invalid syntax (<string>, line 1)

In [8]:
item = devdf.iloc[0]
table = item['table']['table']
question =  item['questions'][4] ['question'] 
gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What is the change in Other in 2019 from 2018? 
 Table: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
{'model_name': 'llama-3.2-3b-instruct', 'model_version': '1', 'text_output': "Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: What is the change in Other in

"def run():\n    # your code here\n    pass\n \n\nNote: The table is a list of lists in python, where each inner list represents a row in the table. The values in the table are strings, except for the 'Other' column which is a number. We need to convert the 'Other' column to float before we can do any calculations. \n\n\ntable = [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]\n \n\n\ndef run():\n    # your code here\n    pass\n \n\n\ntable = [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]\n \n\n\ndef run():\n    # your code here\n    pass\n \n\n\ntable = [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$ 

In [20]:
import time

url = f"http://172.22.214.120:28800/v2/models/{model_name}/generate"

from tqdm.notebook import tqdm as log_progress
etime = 0
call = 0
res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            #code = gen_code(llm, q, table)            
            start = time.time()
            code = gen_code_infer(url, q, table)
            end = time.time()
            etime = etime + (end - start)
        
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, item['table'], err))
            call = call + 1
print (etime/call)

0it [00:00, ?it/s]

<|start_header_id|>system<|end_header_id|>You are an expert on answering questions using an external data table, and arithmetic calculations. You receive a context as a TABLE and a QUESTION. In TABLE numeric values usually have commas as thousand separators. You have to generate a python code, without sample data, which can answer the question. Your response MUST contain only one function called  'run', and no wrapping class.  The function MUST return results with format (value, scale). You ARE NOT ALLOWED to write ANY other words.<|eot_id|><|start_header_id|>user<|end_header_id|>Given the following TABLE: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']], answer the following QUESTION: {'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23', 'order': 5, 'question': 'What is the change in Other in 2019 from 2018?', 'answer': -

In [21]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

special case
special case
special case
0.1452991452991453 0.1452991452991453 0.36752136752136755


In [28]:
import pickle
with open('res_p14_unit.pickle', 'wb') as f:
    pickle.dump(res,f)